In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Priors as hidden mixtures

Adapted from [Unit 7: tasmixture.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/tasmixture.odc) and [Jeremymus.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/Jeremymus.odc)

Associated lecture video: Unit 7 lesson 4

## Student T likelihood with Normal prior vs Normal likelihood with Gamma prior
Not really a problem statement - this is just to demonstrate the equivalence of a couple methods. Probably add notes and explanation here. Variable names from original problem are retained right now, but they are a mess. Will fix later.

Confusingly, the Student's T distribution parameter sigma in PyMC is equivalent to the tau parameter in the BUGS parameterization.

In [2]:
df = 6
x = y = 10
mu0 = 6
tau1 = 10
tau = 0.4
a = df / 2
b = df / (2 * tau)

with pm.Model() as m1:
    mu1 = pm.StudentT("mu1", nu=df, mu=mu0, sigma=tau)
    pm.Normal("X", mu1, tau=tau1, observed=x)

    prec = pm.Gamma("prec", a, b)
    mu2 = pm.Normal("mu2", mu0, tau=prec)
    pm.Normal("Y", mu2, tau=tau1, observed=y)

    trace = pm.sample(3000)

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/multipledispatch/dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]
	[ConstrainedVar, Var, Mapping], [object, ConstrainedVar, Mapping]
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]


Consider making the following additions:

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 5 seconds.


In [3]:
az.summary(trace)

mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
mu1   9.826  0.320   9.258   10.461      0.003    0.002   14798.0    8971.0   
mu2   9.912  0.317   9.313   10.499      0.002    0.002   16325.0    9445.0   
prec  0.232  0.125   0.033    0.460      0.001    0.001   13439.0    8511.0   

      r_hat  
mu1     1.0  
mu2     1.0  
prec    1.0

## Jeremy's IQ - Normal prior as a scale mixture of uniforms

Normal as Scale Mixture of Uniforms

Any Symmetric Unimodal distribution is a Scale Mixture of Uniforms

y|mu, sigma2  ~ N(mu, sigma2) 

is the same as

y|mu, xi  ~ U(mu - sigma * d^0.5,   mu + sigma * d^0.5)

d ~ Ga(3/2, 1/2)

If 
d ~ Ga(3/2, s/2), s<1 heavy tails, s>1 light tails


In [4]:
precy = 0.0125
precth = 0.0083333
mu = 110
s_list = [0.5, 1, 2]
y = 98

summaries = []

for s in s_list:
    beta = s / 2
    with pm.Model() as m2:
        d = pm.Gamma("d", 1.5, beta)
        lb = mu - (d / precth) ** 0.5
        ub = mu + (d / precth) ** 0.5

        theta = pm.Uniform("theta", lb, ub)

        pm.Normal("y", mu=theta, tau=precy, observed=y)

        trace = pm.sample(3000)

    summaries.append(az.summary(trace, kind="stats"))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [d, theta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 10 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [d, theta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 7 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [d, theta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 10 seconds.


In [5]:
for s, summary in zip(s_list, summaries):
    print(f"{s=}")
    print(summary)
    print("\n")

s=0.5
          mean     sd  hdi_3%  hdi_97%
d        5.371  4.430   0.008   13.327
theta  100.843  7.653  86.061  114.692


s=1
          mean     sd  hdi_3%  hdi_97%
d        2.844  2.264   0.016    6.981
theta  102.803  6.929  89.582  115.632


s=2
          mean     sd  hdi_3%  hdi_97%
d        1.521  1.186   0.002    3.679
theta  104.791  5.841  93.827  115.692




In [6]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Fri Feb 03 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

arviz: 0.14.0
pymc : 5.0.1
numpy: 1.24.1

